In [ ]:
from datetime import datetime
import pandas as pd
import requests
import numpy as np
from forex_python.converter import get_rate

In [ ]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance.csv')

In [ ]:
timestamps1 = [int(datetime.strptime(time, '%m/%d/%Y %H:%M').timestamp()*1000) for time in data.UTC_Time]
timestamps2 = [time + 59999 for time in timestamps1]

In [ ]:
i = 10
tick_interval = '1m'

prices = []
markets = []
markets1 = []
prices1 = []
for i in range(len(timestamps1)):
    price1 = '1'
    market1 = 'None'
    print(i)
    if data.Coin[i] not in ['EUR', 'NFT']:
        try:
            market = data.Coin[i]+'EUR'
            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
            price = requests.get(url).json()[0][1]
            
        except:
            try:
                market = 'EUR'+data.Coin[i]
                url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                price = str(1/float(requests.get(url).json()[0][1]))
                market = data.Coin[i]+'EUR'
                
            except:
                try:
                    market = data.Coin[i]+'USDT'
                    market1 = 'EURUSDT'
                    url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price = requests.get(url).json()[0][1]
                    
                    url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price1 = requests.get(url1).json()[0][1]
                
                except:
                    try:
                        market = data.Coin[i]+'BUSD'
                        market1 = 'EURBUSD'
                        url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price = requests.get(url).json()[0][1]

                        url1 = 'https://api.binance.com/api/v3/klines?symbol=EURBUSD&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price1 = requests.get(url1).json()[0][1]
                        
                    except:
                        try:
                            market = data.Coin[i] + 'BTC'
                            market1 = 'EURBTC'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=BTCEUR&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price1 = str(1/float(requests.get(url1).json()[0][1]))
                            
                        except:
                            market = data.Coin[i]+'USDT'
                            market1 = 'EURUSDT'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(requests.get(url).json()[0][0])
                            price1 = requests.get(url1).json()[0][1]

                            print(requests.get(url).json()[0][0])
                            print(requests.get(url1).json()[0][0])
                            print(data.Coin[i])
                            
    elif data.Coin[i] == 'EUR':
        price = '1'
        market = 'EUR'
    elif data.Coin[i] == 'NFT':
        price = '0'
        market = 'NFT'
    prices.append(price)
    prices1.append(price1)
    markets.append(market)
    markets1.append(market1)

In [ ]:
UnitPrices = [float(prices[i]) / float(prices1[i]) for i in range(len(prices))]
totalPrices = [UnitPrices[i] * float(data.Change[i]) for i in range(len(prices)) ]

In [ ]:
data1 = pd.DataFrame({'Unit Prices': UnitPrices, 'Total Price (EUR)': totalPrices})
#'Tool':markets, 'Tool Price':prices, 'Tool to EUR': markets1, 
#                       'Coefficient Price': prices1, 

In [ ]:
Data = pd.concat([data, data1], axis=1)

In [ ]:
Data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Binance Masoud EUR.csv')

In [ ]:
Data=pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Binance Masoud EUR.csv')

In [ ]:
uniqueCoins = np.unique(Data.Coin)
uniqueCoins

In [ ]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data.Change[Data.Coin==i]),10))

In [ ]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data['Total Price (EUR)'][Data.Coin==i]),10))

In [ ]:
Data["UTC_Time"] = pd.to_datetime(Data["UTC_Time"], format='%m/%d/%Y %H:%M')
Data = Data.sort_values(by="UTC_Time")

Data

In [ ]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': Data['Change'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'date_buy': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'price_buy': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'amount_sell': Data['Change'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'date_sell': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'price_sell': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change<0)].to_list()}

In [ ]:
ALL

In [ ]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [ ]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [ ]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)
Report = Report[Report['Sell Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

In [ ]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance_Report.csv')

In [ ]:
Deposits = Data[Data.Operation == 'Deposit'].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Deposits['Total Price (EUR)'] = np.round(Deposits['Total Price (EUR)'], 2)
Deposits['Fee (EUR)'] = [0.0]*len(Deposits['Total Price (EUR)'])
Deposits.iloc[[8, 9, 13, 14], 6] = [0.2, 4.4, 5.57, 13.26]
Deposits.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']
Deposits["Date"] = pd.to_datetime(Deposits["Date"], format='%m/%d/%Y %H:%M')
Deposits = Deposits.sort_values(by="Date").reset_index(drop=True)
Deposits = Deposits[Deposits['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)
Deposits

In [ ]:
Deposits.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance_Deposits.csv')

In [ ]:
Withdraw = Data[Data.Operation.isin(['Withdraw', 'Binance Card Spending'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Unnamed: 0'],axis=1)
Withdraw = Withdraw.fillna('No Fee')
Withdraw['Total Price (EUR)'] = -np.round(Withdraw['Total Price (EUR)'], 2)
Withdraw['Change'] *= -1
Withdraw.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Fee (EUR)', 'Unit Price (EUR)', 'Total Price (EUR)']
Withdraw = Withdraw [['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']]
Withdraw["Date"] = pd.to_datetime(Withdraw["Date"], format='%m/%d/%Y %H:%M')
Withdraw = Withdraw.sort_values(by="Date").reset_index(drop=True)
Withdraw = Withdraw[Withdraw['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Withdraw

In [ ]:
Withdraw.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance_Withdrawals.csv')

In [ ]:
Fee = Data[Data.Operation.isin(['Fee'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Fee['Total Price (EUR)'] = -np.round(Fee['Total Price (EUR)'], 2)
Fee['Change'] *= -1
Fee.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Fee["Date"] = pd.to_datetime(Fee["Date"], format='%m/%d/%Y %H:%M')
Fee = Fee.sort_values(by="Date").reset_index(drop=True)
Fee = Fee[Fee['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)


Fee

In [ ]:
Fee.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance_Fees.csv')

In [ ]:
Cashback = Data[Data.Operation.isin(['Card Cashback', 'Commission Rebate', 'Launchpool Interest'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Cashback['Total Price (EUR)'] = np.round(Cashback['Total Price (EUR)'], 2)
Cashback['Change'] *= -1
Cashback.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Cashback["Date"] = pd.to_datetime(Cashback["Date"], format='%m/%d/%Y %H:%M')
Cashback = Cashback.sort_values(by="Date").reset_index(drop=True)
Cashback = Cashback[Cashback['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Cashback

In [ ]:
Cashback.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Masoud_Binance_Cashbacks.csv')

In [ ]:
np.unique(Data.Operation)

In [ ]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

In [ ]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

In [ ]:
sum(Deposits['Total Price (EUR)'])

In [ ]:
sum(loss)

In [ ]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

In [ ]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

# Etoro

In [ ]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro.csv')

In [ ]:
data

In [ ]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

In [ ]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [ ]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

In [ ]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [ ]:
data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro EUR.csv')

In [ ]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

In [ ]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')

In [ ]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

In [ ]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [ ]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [ ]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [ ]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [ ]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report.csv')

In [ ]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [ ]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [ ]:
Report_Stock = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)

In [ ]:
Report_Crypto = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [ ]:
Report2020.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report 2020.csv')

Report_Stock.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Stock Report 2021.csv')
Report_Crypto.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Crypto Report 2021.csv')


In [ ]:
get_rate("USD", "EUR",datetime(2021,4,13))

In [ ]:
0.8441668073611345*2

In [ ]:
datetime(2021,5,16).day

In [ ]:
2225.57 + 2.76 - 1104.95 - 27.92 

# Crypto.com

In [ ]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\crypto_transactions_record_20220509_073401.csv')

In [ ]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount'],axis=1)
Cashbacks



In [ ]:
sum(Cashbacks['Native Amount (EUR)'])

In [ ]:
Cashbacks.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Cashback.csv')

In [ ]:
data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%m/%d/%Y %H:%M')
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')


data

In [ ]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

In [ ]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [ ]:
ALL

In [ ]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [ ]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [ ]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [ ]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Report.csv')

In [ ]:
ALL

In [ ]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

In [ ]:
260.80-219.15+92.64

In [ ]:
3022.4-1194.71+15.93 - 75.48 


In [53]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

1771.820000000013

In [54]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

1607.670000000001

In [36]:
sum(Deposits['Total Price (EUR)'])

10418.68

In [37]:
sum(loss)

1198.8099999999986

In [38]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

80.84

In [348]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    0.00000
ALICE :    0.00000
BAR :    0.00000
BNB :    0.00285
BTC :    0.00000
BUSD :    0.15344
DOGE :    0.00000
DOT :    0.00000
ENJ :    0.00000
ETH :    0.00000
EUR :    0.00031
FXS :    0.00000
HNT :    0.00000
IOTA :    0.00000
LTC :    0.00000
LUNA :    0.00000
MANA :    0.00000
MATIC :    0.00000
MTL :    0.00000
NFT :    18334.49575
RVN :    0.00000
SFP :    0.00000
SHIB :    0.00000
SOL :    0.00000
STMX :    0.00000
TRX :    0.00000
USDT :    0.00000
WABI :    0.14720
XLM :    0.00000
XRP :    0.00000
ZRX :    0.00000


# Etoro

In [161]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro.csv')

In [162]:
data

,Position ID,Action,Amount,Units,Open Date,Close Date,Leverage,Spread,Profit,Open Rate,Close Rate,Take profit rate,Stop lose rate,Rollover Fees and Dividends,Copied From,Type,ISIN,Notes
0,798900619,Buy Boeing,100.00,0.465875,18/11/2020 15:33:00,01/12/2020 14:49:07,1,0.0,0.22,214.63,215.10,2361.13,0.01,0.0,-,Stocks,US0970231058,NaN
1,798046203,Buy Boeing,100.00,0.478515,17/11/2020 20:20:54,01/12/2020 14:39:45,1,0.0,2.95,208.98,215.14,2298.78,0.01,0.0,-,Stocks,US0970231058,NaN
2,673064189,Buy Amazon.com Inc,333.74,0.099928,13/07/2020 15:05:11,01/12/2020 14:36:05,1,0.0,-17.11,3339.84,3168.62,0.00,0.01,0.0,-,Stocks,US0231351067,NaN
3,799341020,Buy Boeing,50.00,0.243605,18/11/2020 20:32:06,01/12/2020 14:35:17,1,0.0,1.93,205.25,213.19,2257.75,0.01,0.0,-,Stocks,US0970231058,NaN
4,800480812,Buy Boeing,50.00,0.246427,19/11/2020 17:16:56,01/12/2020 14:35:09,1,0.0,2.54,202.90,213.19,2231.90,0.01,0.0,-,Stocks,US0970231058,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,804049275,"Buy Tesla Motors, Inc.",58.28,0.111948,23/11/2020 15:24:57,11/01/2021 20:18:50,1,0.0,32.74,520.60,813.06,5725.50,0.01,0.0,-,Stocks,US88160R1014,NaN
415,804055078,"Buy Tesla Motors, Inc.",50.00,0.096028,23/11/2020 15:26:45,11/01/2021 20:18:50,1,0.0,28.08,520.68,813.06,5727.49,0.01,0.0,-,Stocks,US88160R1014,NaN
416,798030691,"Buy Tesla Motors, Inc.",100.00,0.227433,17/11/2020 20:10:38,11/01/2021 20:18:50,1,0.0,84.92,439.69,813.06,4836.59,0.01,0.0,-,Stocks,US88160R1014,NaN
417,797571688,"Buy Tesla Motors, Inc.",100.00,0.224800,17/11/2020 16:05:11,11/01/2021 20:18:50,1,0.0,82.78,444.84,813.06,4893.24,0.01,0.0,-,Stocks,US88160R1014,NaN


In [158]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [164]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [168]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [173]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [180]:
data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro EUR.csv')

In [198]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_buy['Action'] = ['Buy']*len(data_buy.Instrument)
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [200]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')

In [201]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

array(['AIRBUS SE', 'Abbott Laboratories', 'Advanced Micro Devices Inc',
       'Air France-KLM', 'Alibaba', 'Alphabet', 'Amazon.com Inc',
       'American Airlines Group Inc', 'Apple', 'Aurora Cannabis Inc',
       'Ballard Power Systems Inc.', 'BioNano Genomics Inc', 'Bitcoin',
       'Boeing', 'Booking Holdings Inc', 'Boxlight Corp',
       'Build and Build', 'Cardano', 'Cocrystal Pharma Inc',
       'Delta Air Lines Inc (DE)', 'Dogecoin', 'Ethereum',
       'FuelCell Energy Inc', 'Globalstar', 'IOTA', 'Litecoin',
       'Meta Platforms Inc', 'Microsoft', 'Moderna Inc',
       'NVIDIA Corporation', 'Nio Inc.', 'Palantir Technologies Inc.',
       'Pfizer', 'Plug Power Inc', 'Tesla Motors, Inc.', 'Walmart Inc.',
       'Zoom Video Communications Inc', 'easyJet'], dtype=object)

In [228]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [229]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [230]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [231]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [260]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report.csv')

In [245]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [256]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [257]:
Report_Stock = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)

In [258]:
Report_Crypto = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [259]:
Report2020.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report 2020.csv')

Report_Stock.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Stock Report 2021.csv')
Report_Crypto.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Crypto Report 2021.csv')


In [372]:
get_rate("USD", "EUR",datetime(2021,4,13))

0.8406186953597848

In [359]:
0.8441668073611345*2

1.688333614722269

In [368]:
datetime(2021,5,16).day

16

In [367]:
2225.57 + 2.76 - 1104.95 - 27.92 

1095.4600000000003

# Crypto.com

In [481]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\crypto_transactions_record_20220509_073401.csv')

In [482]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount'],axis=1)
Cashbacks



,Timestamp (UTC),Transaction Description,Currency,Amount,Native Amount (EUR)
0,12/30/2021 16:08,Card Cashback,CRO,0.748426,0.37
1,12/30/2021 15:39,Card Cashback,CRO,0.355138,0.18
2,12/30/2021 15:21,Card Cashback,CRO,0.549071,0.27
3,12/29/2021 16:20,Card Cashback,CRO,0.209790,0.11
4,12/29/2021 15:49,Card Cashback,CRO,0.298225,0.15
...,...,...,...,...,...
122,4/16/2021 6:36,Card Cashback,CRO,1.038671,0.20
123,4/15/2021 6:19,Card Cashback Reversal,CRO,-0.090742,0.02
124,4/14/2021 8:20,Sign-up Bonus Unlocked,CRO,331.014248,20.90
125,4/13/2021 18:37,Referral Bonus Reward,CRO,111.744675,17.59


In [483]:
sum(Cashbacks['Native Amount (EUR)'])

260.799099345

In [496]:
Cashbacks.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Cashback.csv')

In [484]:
data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%m/%d/%Y %H:%M')
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')


data

,UTC_Time,Transaction Description,Coin,Change,Total Price (EUR)
243,2021-01-23 00:31:00,Buy CRO,CRO,1000.000000,60.68
279,2021-01-28 13:30:00,CRO -> BNB,BNB,1.565000,53.88
242,2021-01-28 13:30:00,CRO -> BNB,CRO,-999.882729,53.88
239,2021-02-07 16:19:00,Convert Dust,CRO,5.045851,0.26
240,2021-02-07 16:19:00,Convert Dust,BNB,-0.005000,0.26
...,...,...,...,...,...
4,2021-12-29 15:49:00,Card Cashback,CRO,0.298225,0.15
3,2021-12-29 16:20:00,Card Cashback,CRO,0.209790,0.11
2,2021-12-30 15:21:00,Card Cashback,CRO,0.549071,0.27
1,2021-12-30 15:39:00,Card Cashback,CRO,0.355138,0.18


In [485]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

array(['ADA', 'BNB', 'BOSON', 'BTC', 'CRO', 'DOGE', 'DOT', 'EFI', 'ENJ',
       'ETH', 'EUR', 'HBAR', 'LTC', 'SHIB', 'USDC', 'XLM', 'XRP'],
      dtype=object)

In [486]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [487]:
ALL

{'ADA': {'amount_buy': [84.0, 100.098, 25.896, 0.094],
  'date_buy': [Timestamp('2021-02-07 23:23:00'),
   Timestamp('2021-09-02 06:04:00'),
   Timestamp('2021-09-02 06:31:00'),
   Timestamp('2021-09-08 07:27:00')],
  'price_buy': [45.03, 256.39, 66.4, 0.18],
  'amount_sell': [-84.0, -125.994, -0.094],
  'date_sell': [Timestamp('2021-02-08 22:41:00'),
   Timestamp('2021-09-08 07:15:00'),
   Timestamp('2021-09-08 07:27:00')],
  'price_sell': [-47.01, -243.52, -0.18]},
 'BNB': {'amount_buy': [1.565, 0.82],
  'date_buy': [Timestamp('2021-01-28 13:30:00'),
   Timestamp('2021-02-07 23:14:00')],
  'price_buy': [53.88, 45.93],
  'amount_sell': [-0.005, -1.56, -0.82],
  'date_sell': [Timestamp('2021-02-07 16:19:00'),
   Timestamp('2021-02-07 16:19:00'),
   Timestamp('2021-02-07 23:21:00')],
  'price_sell': [-0.26, -83.13, -45.77]},
 'BOSON': {'amount_buy': [0.07722495, 0.12870825],
  'date_buy': [Timestamp('2021-09-08 07:38:00'),
   Timestamp('2021-10-08 12:16:00')],
  'price_buy': [0.08, 0.12

In [488]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [489]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [490]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [491]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Report.csv')

In [492]:
ALL

{'ADA': {'amount_buy': [],
  'date_buy': [],
  'price_buy': [],
  'amount_sell': [],
  'date_sell': [],
  'price_sell': []},
 'BNB': {'amount_buy': [],
  'date_buy': [],
  'price_buy': [],
  'amount_sell': [],
  'date_sell': [],
  'price_sell': []},
 'BOSON': {'amount_buy': [],
  'date_buy': [],
  'price_buy': [],
  'amount_sell': [],
  'date_sell': [],
  'price_sell': []},
 'BTC': {'amount_buy': [],
  'date_buy': [],
  'price_buy': [],
  'amount_sell': [],
  'date_sell': [],
  'price_sell': []},
 'CRO': {'amount_buy': [766.2903458550002,
   0.09074157,
   111.7446755,
   331.0142475,
   1.03867114,
   2.02612908,
   4.1773373,
   2.21349313,
   127.962894,
   2.05729728,
   0.10501876,
   3.70697193,
   0.61835914,
   0.15483045,
   0.10332754,
   6.85679025,
   83.34783018,
   0.44285581,
   9.48650135,
   9.31271439,
   168.5158481,
   9.15375442,
   8.71553534,
   34.80059695,
   3.95281245,
   2.61694837,
   6.38155218,
   5.1150662,
   4.34756472,
   16.97949023,
   133.3634115,


In [495]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    0.00000
BNB :    0.00000
BOSON :    0.00000
BTC :    0.00000
CRO :    3238.76151
DOGE :    0.00000
DOT :    0.00000
EFI :    0.00886
ENJ :    0.00000
ETH :    0.00000
EUR :    0.00000
HBAR :    0.19024
LTC :    0.00000
SHIB :    0.00000
USDC :    0.00000
XLM :    0.00000
XRP :    0.00000


In [497]:
260.80-219.15+92.64

134.29000000000002

In [56]:
3022.4-1194.71+15.93 - 75.48 


1768.14